In [1]:
# The OpenAI SDK was updated on Nov 8, 2023 with new guidance for migration
# See: https://github.com/openai/openai-python/discussions/742

## Updated
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  azure_endpoint=os.environ['AZURE_OPENAI_ENDPOINT'],
  api_version = "2024-06-01"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

In [28]:
def get_completion(system_prompt, user_prompt):
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]       
    response = client.chat.completions.create(   
        model=deployment,                                         
        messages=messages,
        temperature=0.5, # this is the degree of randomness of the model's output
        max_tokens=1024
    )

        # Get the token usage  
    completion_tokens= response.usage.completion_tokens
    prompt_tokens = response.usage.prompt_tokens
    total_tokens = response.usage.total_tokens
  
    print(f"Total Tokens: {total_tokens}")  
    print(f"Prompt Tokens: {prompt_tokens}")  
    print(f"Completion Tokens: {completion_tokens}")  
    return response.choices[0].message.content


## Example 1
Analysis with made up data

In [29]:
import pandas as pd
forecast_df = pd.DataFrame({'Record_ID':[1,2,3,4,5],
                            'Unit_Cost':[10,20,30,40,50],
                            'Number of Units':[10,80,8,5,8]})
forecast_df['Total Cost'] =forecast_df['Unit_Cost']*forecast_df['Number of Units']

actuals_df = pd.DataFrame({'Record_ID':[1,2,3,4,5],
                            'Unit_Cost':[8,22,25,42,48],
                            'Number of Units':[8,70,9,7,9]})

actuals_df['Total Cost'] = actuals_df['Unit_Cost']*actuals_df['Number of Units']

In [19]:
forecast_df.head()

,Record_ID,Unit_Cost,Number of Units,Total Cost
0,1,10,10,100
1,2,20,80,1600
2,3,30,8,240
3,4,40,5,200
4,5,50,8,400


In [20]:
actuals_df.head()

,Record_ID,Unit_Cost,Number of Units,Total Cost
0,1,8,8,64
1,2,22,70,1540
2,3,25,9,225
3,4,42,7,294
4,5,48,9,432


In [32]:
def get_compressed_json(df):
    # Flatten the DataFrame  
    df_flattened = pd.json_normalize(df.to_dict(orient='records'))  
    
    # Convert to JSON  
    json_data = df_flattened.to_json(orient='records') 
    return json_data 


def get_prompts(forecast_df, actuals_df, compress=False):

    ## ---------- Call the helper method
    if compress:
        forecast_data = get_compressed_json(forecast_df)
        actuals_data = get_compressed_json(actuals_df)
    else:
        forecast_data = forecast_df.to_dict()
        actuals_data = actuals_df.to_dict()

    ### 1. Set primary content or prompt text
    system_prompt= f"""
    You are a financial analysis assistant.
    You help by comparing actual and forecast data for financial data, in detail and summarizing results. 
    """

    ### 2. Use that in the prompt template below
    user_prompt = f"""
    There are a set of expenses that are compiled under the travel and intertainment General Ledge account.
    These expenses are composed of unit costs and number of units.

    There is a forecast (also known as budget) of each item with the following values:
    {forecast_data}

    Similarly, actual results were reported with the following values:
    {actuals_data}

    Provide a brief analysis of the variance between actuals and forecast, highlighting the main causes of the variance.
    Call out specific percentages or dollar values.
    """
    return system_prompt, user_prompt

In [33]:
# Getting response without compressing data
system_prompt, user_prompt = get_prompts(forecast_df, actuals_df, compress=False)
response = get_completion(system_prompt,user_prompt)
print(response)

Total Tokens: 1157
Prompt Tokens: 420
Completion Tokens: 737
### Variance Analysis: Travel and Entertainment Expenses

#### Overview
The analysis compares the forecasted budget against the actual expenses incurred for travel and entertainment. The key metrics analyzed include Unit Cost, Number of Units, and Total Cost.

#### Summary of Forecast vs. Actual

| Record_ID | Forecast Unit Cost | Actual Unit Cost | Forecast Units | Actual Units | Forecast Total Cost | Actual Total Cost | Variance Total Cost | Variance % |
|-----------|---------------------|------------------|----------------|--------------|---------------------|-------------------|---------------------|------------|
| 1         | $10                 | $8               | 10             | 8            | $100                | $64               | -$36                | -36%       |
| 2         | $20                 | $22              | 80             | 70           | $1600               | $1540             | -$60                |

In [34]:
# Getting response compressing data
system_prompt, user_prompt = get_prompts(forecast_df, actuals_df, compress=True)
response = get_completion(system_prompt,user_prompt)
print(response)

Total Tokens: 1086
Prompt Tokens: 368
Completion Tokens: 718
### Variance Analysis: Travel and Entertainment Expenses

#### Forecast vs. Actual Summary

| Record_ID | Forecast Unit Cost | Forecast Units | Forecast Total Cost | Actual Unit Cost | Actual Units | Actual Total Cost | Variance (Total Cost) | Variance (%) |
|-----------|--------------------|----------------|---------------------|------------------|--------------|-------------------|-----------------------|--------------|
| 1         | $10                | 10             | $100                | $8               | 8            | $64               | -$36                  | -36.0%       |
| 2         | $20                | 80             | $1600               | $22              | 70           | $1540             | -$60                  | -3.75%       |
| 3         | $30                | 8              | $240                | $25              | 9            | $225              | -$15                  | -6.25%       |
| 4         

## Example 2
Using template from Microsoft found here: https://create.microsoft.com/en-us/template/business-expense-budget-d14f83e2-f49d-4080-9e95-0bd4849fa63f


In [35]:
forecast_workbook_path = 'files/Business expense budget.xlsx'
actuals_workbook_path = 'files/Business expense budget.xlsx'
forecast_sheet_name = 'Forecast Expenses Database'
actuals_sheet_name = 'Actuals Expenses Database'


forecast_df = pd.read_excel(forecast_workbook_path, sheet_name=forecast_sheet_name)
actuals_df = pd.read_excel(actuals_workbook_path, sheet_name=actuals_sheet_name)
print(forecast_df.head())
print(actuals_df.head())

         Category Sucategory Time Period  Expense
0  Employee costs      Wages         Jan    85000
1  Employee costs      Wages         Feb    85000
2  Employee costs      Wages         Mar    85000
3  Employee costs      Wages         Apr    87500
4  Employee costs      Wages         May    87500
         Category Sucategory Time Period       Expense
0  Employee costs      Wages         Jan  84824.384249
1  Employee costs      Wages         Feb  84938.026249
2  Employee costs      Wages         Mar  85035.009233
3  Employee costs      Wages         Apr  87463.619793
4  Employee costs      Wages         May  87263.353172


In [36]:
system_prompt, user_prompt = get_prompts(forecast_df, actuals_df,compress=False)
response = get_completion(system_prompt,user_prompt)
print(response)

Total Tokens: 12392
Prompt Tokens: 11916
Completion Tokens: 476
### Variance Analysis: Travel and Entertainment Expenses

#### Summary of Forecast vs. Actual Expenses
The forecasted and actual expenses for the Travel and Entertainment category have been compared across various subcategories. Below are the key findings highlighting the variances in both dollar amounts and percentages.

#### Total Expenses
- **Total Forecasted Expenses**: $1,547,130
- **Total Actual Expenses**: $1,547,783
- **Variance**: $653 (0.04% over budget)

#### Breakdown by Major Categories
1. **Employee Costs**
   - **Forecast**: $1,067,000
   - **Actual**: $1,066,740
   - **Variance**: -$260 (-0.02% under budget)

2. **Office Costs**
   - **Forecast**: $288,090
   - **Actual**: $287,938
   - **Variance**: -$152 (-0.05% under budget)

3. **Training/Travel**
   - **Forecast**: $192,040
   - **Actual**: $192,105
   - **Variance**: +$65 (0.03% over budget)

4. **Totals**
   - **Forecast**: $1,547,130
   - **Actual**

In [37]:
system_prompt, user_prompt = get_prompts(forecast_df, actuals_df,compress=True)
response = get_completion(system_prompt,user_prompt)
print(response)

Total Tokens: 10047
Prompt Tokens: 9350
Completion Tokens: 697
### Variance Analysis: Actual vs. Forecast Expenses

#### Summary of Total Expenses
- **Total Forecasted Expenses for the Year**: $1,542,130
- **Total Actual Expenses for the Year**: $1,518,540.82
- **Variance**: $23,589.18 (1.53% under budget)

### Detailed Analysis by Category

#### 1. **Employee Costs**
- **Forecast**: $1,355,090
- **Actual**: $1,354,371.55
- **Variance**: $718.45 (0.05% under budget)

**Key Points**:
- **Wages**: Forecasted at $1,067,000 vs. Actual $1,066,433.10, a variance of $566.90 (0.05%).
- **Benefits**: Forecasted at $288,090 vs. Actual $287,938.46, a variance of $151.54 (0.05%).
  
**Cause of Variance**: Minor adjustments in both wages and benefits led to negligible variances.

#### 2. **Office Costs**
- **Forecast**: $139,040
- **Actual**: $140,649.38
- **Variance**: -$1,609.38 (1.16% over budget)

**Key Points**:
- **Office Lease**: Forecasted at $117,600 vs. Actual $118,282.67, a variance of $